In [2]:
#Import All Important Libraries
import librosa
from pydub import AudioSegment
import subprocess
import soundfile
import os, glob, pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from pathlib import Path
from sklearn.metrics import accuracy_score, confusion_matrix

In [18]:
# import shutil
# for folder in glob.glob('./Datasets/Actor_*'):
#     for file in glob.glob(folder + '/*.wav'):
#         file1 = file[20:]
#         # shutil.copy(file, './Datasets/Data/' + file1)
#         bashCommand = f"cp -f {file} ./Datasets/Data/{file1}"
#         process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)

In [30]:
#function for extracting mfcc, chroma, and mel features from sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            stft=np.abs(librosa.stft(X))
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [31]:
#Define the motions dictionary
emotions = {
    '01':'neutral',
    '02':'calm',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

#Emotions we want to observe
observed_emotions = ['calm', 'happy', 'fearful', 'disgust']

In [36]:
#Load the data and extract features for each sound file
def load_data(test_size = 0.2):
    x, y = [], []
    for folder in glob.glob('./Datasets/Actor_*'):
        # print(0)
        print(folder)
        for file in glob.glob(folder + '/*.wav'):
            # print(file)
            file_name = os.path.basename(file)
            emotion = emotions[file_name.split('-')[2]]
            if emotion not in observed_emotions:
                continue
            feature = extract_feature(file, mfcc = True, chroma = True, mel = True)
            x.append(feature)
            y.append(emotion)
    return train_test_split(np.array(x), y, test_size = test_size, random_state = 9)

In [37]:
x_train,x_test,y_train,y_test=load_data(test_size=0.3)

./Datasets/Actor_20


ParameterError: Invalid shape for monophonic audio: ndim=2, shape=(209809, 2)

In [ ]:
#Shape of train and test set and Number of features extracted
print((x_train.shape[0], x_test.shape[0]))
print(f'Features extracted: {x_train.shape[1]}')

(386, 166)
Features extracted: 180


In [ ]:
#Initialise Multi Layer Perceptron Classifier
model = MLPClassifier(alpha = 0.01, batch_size = 256, epsilon = 1e-08, hidden_layer_sizes = (1000,), learning_rate = 'adaptive', max_iter = 500)

In [ ]:
history

In [ ]:
#Predict for the test set
y_pred = model.predict(x_test)

In [ ]:
loss_plot=model.loss_curve_
plt.plot(loss_plot)
plt.show()